# HW04

```yaml
Course:   DS 5001
Module:   04 Homework
Topic:    NLP and the Pipeline
Author:   Ryan Lipps
Date:     10 February 2023
```

## Setup

In [13]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
import configparser
import sys
from textparser import TextParser

In [14]:
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']

In [15]:
source_files = f'{data_home}/gutenberg/eliot-set'
data_prefix = 'eliot-set'

In [16]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']

In [17]:
sys.path.append(local_lib)

In [18]:
clip_pats = [
    r"\*\*\*\s*START OF",
    r"\*\*\*\s*END OF"
]

# All are 'chap'and 'm'
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
ohco_pat_list = [
    (507,   rf"^\s*Chapter\s+{roman}\s*$"),
    (145,   rf"^[A-Z]+\s?{roman}*\.$"),
    (6688,  rf"^\s*Chapter\s+{roman}\.\s*$")
]

In [19]:
source_file_list = sorted(glob(f"{source_files}/*.*"))

In [20]:
source_file_list

['/Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/eliot-set/ELIOT_GEORGE_ADAM_BEDE-pg507.txt',
 '/Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/eliot-set/ELIOT_GEORGE_MIDDLEMARCH-pg145.txt',
 '/Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/eliot-set/ELIOT_GEORGE_THE_MILL_ON_THE_FLOSS-pg6688.txt']

In [21]:
book_data = []
for source_file_path in source_file_list:
    book_id = int(source_file_path.split('-')[-1].split('.')[0].replace('pg',''))
    book_title = source_file_path.split('/')[-1].split('-')[0].replace('_', ' ')
    book_data.append((book_id, source_file_path, book_title))

## Build LIB frame

In [22]:
LIB = pd.DataFrame(book_data, columns=['book_id','source_file_path','raw_title'])\
    .set_index('book_id').sort_index()
LIB

,source_file_path,raw_title
book_id,,
145,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE MIDDLEMARCH
507,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE ADAM BEDE
6688,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE THE MILL ON THE FLOSS


In [23]:
try:
    LIB['author'] = LIB.raw_title.apply(lambda x: ', '.join(x.split()[:2]))
    LIB['title'] = LIB.raw_title.apply(lambda x: ' '.join(x.split()[2:]))
    LIB = LIB.drop('raw_title', axis=1)
except AttributeError:
    pass

In [24]:
LIB

,source_file_path,author,title
book_id,,,
145,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,"ELIOT, GEORGE",MIDDLEMARCH
507,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,"ELIOT, GEORGE",ADAM BEDE
6688,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,"ELIOT, GEORGE",THE MILL ON THE FLOSS
